In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory

import os
i = 1
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(i,'- ',os.path.join(dirname, filename))
        i = i + 1


1 -  /kaggle/input/brain-tumor-segmentation/images/2664.png
2 -  /kaggle/input/brain-tumor-segmentation/images/2539.png
3 -  /kaggle/input/brain-tumor-segmentation/images/1231.png
4 -  /kaggle/input/brain-tumor-segmentation/images/1017.png
5 -  /kaggle/input/brain-tumor-segmentation/images/2437.png
6 -  /kaggle/input/brain-tumor-segmentation/images/2015.png
7 -  /kaggle/input/brain-tumor-segmentation/images/2300.png
8 -  /kaggle/input/brain-tumor-segmentation/images/2673.png
9 -  /kaggle/input/brain-tumor-segmentation/images/2823.png
10 -  /kaggle/input/brain-tumor-segmentation/images/1522.png
11 -  /kaggle/input/brain-tumor-segmentation/images/2064.png
12 -  /kaggle/input/brain-tumor-segmentation/images/641.png
13 -  /kaggle/input/brain-tumor-segmentation/images/1088.png
14 -  /kaggle/input/brain-tumor-segmentation/images/173.png
15 -  /kaggle/input/brain-tumor-segmentation/images/2851.png
16 -  /kaggle/input/brain-tumor-segmentation/images/2448.png
17 -  /kaggle/input/brain-tumor-seg

In [2]:
from tensorflow import keras
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model



# Define the architecture of a convolution block
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x


# Define the architecture of the encoder block
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


# Define the architecture of the decoder block
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x
    
# Build the UNET architectre
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    # print(s1.shape, s2.shape, s3.shape, s4.shape)
    # print(p1.shape, p2.shape, p3.shape, p4.shape)
    
    b1 = conv_block(p4, 1024)
    
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    # Define the output layer
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
    input_shape = (256, 256, 3)     # (height, width, channels) for an RGB image
    model = build_unet(input_shape)
    model.summary()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

In [3]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from keras import backend as K

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = keras.layers.Flatten()(y_true)
    y_pred = keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2 as cv
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


""" Global parameters"""
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    
    split_size = int(len(images) * split)
    
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)
    
    train_x, test_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def read_image(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_COLOR)
    x = cv.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x


def read_mask(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_GRAYSCALE)        # (height, width)
    x = cv.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)                  # (height, width, 1)
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y



def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42) 
    tf.random.set_seed(42)
    
    """ Directory for storing files """
    create_dir("files")
    
    """ Hyperparameters """
    batch_size = 16
    lr = 0.00058 #0.075 #1e-4
    num_epochs = 150
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "log.csv")
    
    """ Dataset """
    dataset_path = "/kaggle/input/brain-tumor-segmentation"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")
    
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
    
    for x, y in train_dataset:
        print(x.shape, y.shape)
    
    
    """ Model """
    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef])
    
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False),
    ]
    
    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Train: 2452 - 2452
Valid: 612 - 612
Test: 612 - 612
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 256, 3) (16, 256, 256, 1)
(16, 256, 25

In [5]:
# Testing the model for test data
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2 as cv
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from sklearn.model_selection import train_test_split



""" Global parameters"""
H = 256
W = 256


""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255
    
    line = np.ones((H, 10, 3)) * 255
    
    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv.imwrite(save_image_path, cat_images)
    
    ## print(image.shape, mask.shape, y_pred.shape)
    

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)
    
    """ Directory for storing files """
    create_dir("results")
    
    """ Load the model """
    with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
        model = keras.models.load_model(os.path.join("files", "model.h5"))
        
    """ Dataset """
    dataset_path = "/kaggle/input/brain-tumor-segmentation"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    
    
    """ Prediction and Evaluation """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        
        """ Extracting the name """
        name = x.split("/")[-1]   
        
        """ Reading the image """
        image = cv.imread(x, cv.IMREAD_COLOR)  ## [H, W, 3]
        image = cv.resize(image, (W, H))
        x = image / 255.0
        x = np.expand_dims(x, axis=0)          ## [1, H, W, 3]
        
        """ Reading the mask """
        mask = cv.imread(y, cv.IMREAD_GRAYSCALE)
        mask = cv.resize(mask, (W, H))
        
        """ Prediction """
        y_pred = model.predict(x, verbose=0)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)
        
        """ Saving the prediction """
        save_image_path = os.path.join("results", name)
        save_results(image, mask, y_pred, save_image_path)
        
        
        """ Flatten the array """
        mask = mask / 255.0
        mask = (mask > 0.5).astype(np.int32).flatten()
        y_pred = y_pred.flatten()
        
        """ Calculating the metrics values """
        f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, f1_value, jac_value, recall_value, precision_value])
        
    
    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"F1: {score[0]:0.5f}")
    print(f"Jaccard: {score[1]:0.5f}")
    print(f"Recall: {score[2]:0.5f}")
    print(f"Precision: {score[3]:0.5f}")
    
    df = pd.DataFrame(SCORE, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

  1%|          | 4/612 [00:01<03:14,  3.13it/s]/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  3%|▎         | 20/612 [00:04<01:39,  5.95it/s]/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  7%|▋         | 43/612 [00:08<01:43,  5.49it/s]/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

F1: 0.76955
Jaccard: 0.68957
Recall: 0.77521
Precision: 0.80463
